In [ ]:
import torch

# 1. Check available device and assign it to the 'device' variable
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Load the model, mapping its saved tensors to the determined device
model = torch.jit.load(r"ghost_netv1.pt", map_location=device)

# 3. Explicitly move the entire model structure (including all parameters) 
#    to the determined device. (This is generally done right after loading 
#    to ensure everything is properly on the GPU.)
model.to(device) 

# Verify the model is on the GPU
print(f"Model parameters device: {next(model.parameters()).device}")

In [ ]:
feature_layers = []
for name, module in model.named_children():
    if name != "classifier":
        feature_layers.append(module)
    else:
        break
feature_extractor = torch.nn.Sequential(*feature_layers)

In [ ]:
from tqdm import tqdm

# Set the feature extractor to evaluation mode (essential!)
feature_extractor.eval() 

feature_vectors = []

with torch.no_grad(): # Disable gradient calculation for efficiency
    for images, _ in tqdm(train_loader, desc="Extracting Features"):
        images = images.to(device)
        
        # 1. Get the raw output from the Feature Extractor
        vector_output = feature_extractor(images)
        
        # 2. Flatten the output into a single feature vector per image
        # This step depends on the last layer of your feature_extractor. 
        # If the last layer is GlobalAvgPool2d, the output is (BatchSize, Feature_Dim).
        # If it's a Conv layer, you need to flatten:
        # We assume the output is already the embedding (BatchSize, Feature_Dim)
        
        # A safer flatten for JIT/Scripted models if you don't know the final layer:
        # Example: If GhostNet's final feature layer is 1280 channels:
        # vector = vector_output.view(vector_output.size(0), -1) 
        
        # Assuming the final layer outputs the flat feature vector (e.g., after AdaptiveAvgPool2d):
        vectors = vector_output.squeeze() 

        feature_vectors.append(vectors.cpu())

# Concatenate all feature vectors into one large tensor
normal_features = torch.cat(feature_vectors, dim=0)
feature_dim = normal_features.shape[1] 
print(f"Total Normal Features Generated: {normal_features.shape}")
print(f"Feature Vector Dimension: {feature_dim}")

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define the Autoencoder (Encoder + Decoder)
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim=128):
        super(Autoencoder, self).__init__()
        
        # Encoder (Compresses the feature vector)
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, input_dim // 2),
            nn.ReLU(True),
            nn.Linear(input_dim // 2, latent_dim), # Bottleneck
            nn.ReLU(True)
        )
        
        # Decoder (Reconstructs the feature vector)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, input_dim // 2),
            nn.ReLU(True),
            nn.Linear(input_dim // 2, input_dim),
            # No activation on the final layer for reconstruction (allows negative values)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# --- Setup and Training ---

# Hyperparameters (Adjust these)
LATENT_DIM = 256 # A smaller dimension than feature_dim
NUM_EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 1e-3

# Initialize the Autoencoder
ae_model = Autoencoder(input_dim=feature_dim, latent_dim=LATENT_DIM).to(device)

# Loss Function: Mean Squared Error (for L2 reconstruction loss)
criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(ae_model.parameters(), lr=LEARNING_RATE)

# Create a DataLoader for the features themselves
feature_dataset = torch.utils.data.TensorDataset(normal_features.to(device))
feature_loader = torch.utils.data.DataLoader(feature_dataset, batch_size=BATCH_SIZE, shuffle=True)


print("\nStarting Autoencoder Training...")

for epoch in range(NUM_EPOCHS):
    total_loss = 0
    for (data,) in feature_loader:
        # data is the input feature vector, and the target is also data (self-supervision)
        
        optimizer.zero_grad()
        
        reconstructed_data = ae_model(data)
        
        # Loss is the difference (reconstruction error)
        loss = criterion(reconstructed_data, data) 
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * data.size(0)
    
    avg_loss = total_loss / len(normal_features)
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {avg_loss:.6f}")

# Save the trained Autoencoder model
torch.save(ae_model.state_dict(), 'autoencoder_anomaly_detector.pth')
print("Autoencoder model saved.")

In [44]:
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os
import pickle
from PIL import Image
from tqdm import tqdm
import sys

# --- Configuration ---
# 🎯 SET YOUR ABSOLUTE PATHS HERE 🎯
LOCAL_PKL_PATH = r"C:\Users\ch0kism\Documents\GitHub\Supparod_MLOPS\processed_data\preprocess_artifact.pkl"
# 🎯 YOU MUST SET THE PATH TO YOUR CSVs (if they are also local) 🎯
# Assume your CSVs are in a directory named 'validation_data' next to your code root, 
# or wherever your data validation step put them.
LOCAL_DATA_ROOT = r"C:\Users\ch0kism\Documents\GitHub\Supparod_MLOPS\validation_data" 
IMAGE_DATA_ROOT = r"C:\Users\ch0kism\Documents\GitHub\Supparod_MLOPS\Dataset_github"

BATCH_SIZE = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- CustomDataset Class Definition (Re-pasted from your file for completeness) ---
class CustomDataset(Dataset):
    def __init__(self, path_label, transform=None):
        self.path_label = path_label
        self.transform = transform
    def __len__(self):
        return len(self.path_label)
    def __getitem__(self, idx):
        path, label = self.path_label[idx]
        img = Image.open(path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        label = torch.tensor(label, dtype=torch.long)
        return img, label
    
def create_path_label_list(df, image_root_path=IMAGE_DATA_ROOT):
    path_label_list = []
    for _, row in df.iterrows():
        path_label_list.append((row['file_path'], row['label']))
    return path_label_list
# ---------------------------------------------------------------------------------

# --- 1. LOCAL ARTIFACT AND DATA LOADING ---
try:
    print("Loading preprocessing artifacts from local PKL file...")
    # Load the Label Encoder and the Inference Transform from the PKL file
    with open(LOCAL_PKL_PATH, "rb") as f:
        artifact_data = pickle.load(f)

    le = artifact_data["label_encoder"]
    inference_transform = artifact_data["transform"]
    
    # 1.2 Read the training data CSV (assuming it's available locally)
    print(f"Loading training data from: {LOCAL_DATA_ROOT}")
    train_path = os.path.join(LOCAL_DATA_ROOT, "train.csv")
    train_df = pd.read_csv(train_path)
    
    # 1.3 Re-encode labels using the loaded LabelEncoder
    # Note: If 'class' column is not in train_df, this will fail.
    train_df['label'] = le.transform(train_df['class'])
    
    # 1.4 Create the DataLoader

    train_dataset = CustomDataset(create_path_label_list(train_df), image_root_path=IMAGE_DATA_ROOT, transform=inference_transform)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    print(f"✅ train_loader successfully created with {len(train_loader.dataset)} samples.")

except FileNotFoundError as e:
    print(f"FATAL: File not found. Check your paths: {e}")
    print("HINT: Ensure LOCAL_PKL_PATH and LOCAL_DATA_ROOT are set correctly.")
    sys.exit(1)
except Exception as e:
    print(f"FATAL: Error during data setup: {e}")
    sys.exit(1)

Loading preprocessing artifacts from local PKL file...
Loading training data from: C:\Users\ch0kism\Documents\GitHub\Supparod_MLOPS\validation_data
FATAL: Error during data setup: CustomDataset.__init__() got an unexpected keyword argument 'image_root_path'


SystemExit: 1

c:\Users\ch0kism\miniconda3\envs\mlflow\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [29]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder 
from torchvision import transforms
import os
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn

import mlflow
from mlflow.artifacts import download_artifacts
from torchvision import transforms
import json
import sys
from PIL import Image

import pickle

In [20]:
try:
    # local_artifact_path = download_artifacts(
    #     run_id=data_validation_id,
    #     artifact_path="validation_data"
    # )
    # print(f"Artifacts downloaded to: {local_artifact_path}")


    # 1.2 สร้างพาธไปยังไฟล์ CSV ที่ดาวน์โหลดมา
    # train_path = os.path.join(local_artifact_path, "train.csv")
    # val_path = os.path.join(local_artifact_path, "val.csv")
    
    # 1.3 อ่านไฟล์ CSV จาก local path ที่ถูกต้อง
    train_df = pd.read_csv(r"validation_data\train.csv")
    val_df = pd.read_csv(r"validation_data\val.csv")
    print("Successfully loaded data from downloaded artifacts.")
    # --- END: โค้ดส่วนที่แก้ไข ---
except Exception as e:
    print(f"Error loading artifacts: {e}")
    print("Please ensure the preprocessing_run_id is correct.")
    sys.exit(1)

le = LabelEncoder()

merged_df = pd.concat([train_df, val_df], ignore_index=True)
le.fit(merged_df['class'])
del merged_df
train_df['label'] = le.transform(train_df['class'])
val_df['label'] = le.transform(val_df['class'])

def create_path_label_list(df):
    path_label_list = []
    for _, row in df.iterrows():
        path = row['file_path']
        label = row['label']
        path_label_list.append((path, label))
    return path_label_list

transform=transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize(224),             # resize shortest side to 224 pixels
        transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                            [0.229, 0.224, 0.225])
])

inference_transform = transforms.Compose([
transforms.Resize(224),
transforms.CenterCrop(224),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])



class CustomDataset(Dataset):
    def __init__(self, path_label, transform=None):
        self.path_label = path_label
        self.transform = transform

    def __len__(self):
        return len(self.path_label)

    def __getitem__(self, idx):
        path, label = self.path_label[idx]
        img = Image.open(path).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        label = torch.tensor(label, dtype=torch.long)  #LongTensor
        return img, label
    
batch_size = 64
train_dataset = CustomDataset(create_path_label_list(train_df), transform=transform)
val_dataset = CustomDataset(create_path_label_list(val_df), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Successfully loaded data from downloaded artifacts.


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
MODEL_PATH = r"ghost_netv1.pt" 

try:
    print("\n--- 2. Loading Model and Creating Feature Extractor ---")
    
    # Load model and map to the determined device
    model = torch.jit.load(MODEL_PATH, map_location=device)
    model.to(device) 
    
    # Create the Feature Extractor (removes the classifier/final layer)
    feature_layers = []
    for name, module in model.named_children():
        if name != "classifier":
            feature_layers.append(module)
        else:
            break
            
    feature_extractor = torch.nn.Sequential(*feature_layers)
    
    # Verify the model is on the correct device
    print(f"✅ GhostNet model loaded and on device: {next(model.parameters()).device}")

except Exception as e:
    print(f"FATAL: Model loading or Feature Extractor setup failed. Error: {e}")
    sys.exit(1)


--- 2. Loading Model and Creating Feature Extractor ---
✅ GhostNet model loaded and on device: cuda:0


In [ ]:
# --- 3. Feature Extraction Loop ---

feature_extractor.eval()
feature_vectors = []

print("\n--- 3. Extracting Normal Feature Vectors ---")
with torch.no_grad():
    for images, _ in tqdm(train_loader, desc="Extracting Features"):
        images = images.to(device)
        
        vector_output = feature_extractor(images)
        
        # FLATTEN: Reshape [N, C, H, W] to [N, C * H * W] 
        # This is the expected operation after the convolutional body.
        vectors = vector_output.view(vector_output.size(0), -1) 

        feature_vectors.append(vectors.cpu())

normal_features = torch.cat(feature_vectors, dim=0)
feature_dim = normal_features.shape[1] 
print(f"✅ Feature Extraction Complete. Total Vectors: {normal_features.shape}")
print(f"Feature Vector Dimension (Input/Output size for AE): {feature_dim}")


--- 3. Extracting Normal Feature Vectors ---


Extracting Features: 100%|██████████| 329/329 [01:39<00:00,  3.30it/s]

✅ Feature Extraction Complete. Total Vectors: torch.Size([21000, 1280])
Feature Vector Dimension (Input/Output size for AE): 1280


In [30]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim=256):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, input_dim // 2), nn.ReLU(True),
            nn.Linear(input_dim // 2, latent_dim), nn.ReLU(True)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, input_dim // 2), nn.ReLU(True),
            nn.Linear(input_dim // 2, input_dim) # Final output is linear
        )
    def forward(self, x):
        return self.decoder(self.encoder(x))

# Hyperparameters
LATENT_DIM = 256 
NUM_EPOCHS = 200
# OPTIMIZATION 1: Increased Batch Size (Leveraging 16GB VRAM)
BATCH_SIZE_AE = 512 # Increased from 128 to 512 (or higher, e.g., 1024, if memory allows)
LEARNING_RATE = 1e-4

# Setup
# OPTIMIZATION 2: Move normal_features to GPU ONCE
# This is cleaner than doing it within the TensorDataset definition.
normal_features_gpu = normal_features.to(device)

ae_model = Autoencoder(input_dim=feature_dim, latent_dim=LATENT_DIM).to(device)
criterion = nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(ae_model.parameters(), lr=LEARNING_RATE)

# Data Loader
feature_dataset = TensorDataset(normal_features_gpu) # Use the GPU tensor
feature_loader = DataLoader(feature_dataset, batch_size=BATCH_SIZE_AE, shuffle=True)

print("\n--- 4. Training Autoencoder for Anomaly Detection ---")
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    ae_model.train()
    # OPTIMIZATION 3: Use tqdm for clear progress monitoring
    for (data,) in tqdm(feature_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        # NOTE: 'data' is already on the GPU because 'normal_features_gpu' was used.
        # No need for data.to(device) inside the loop.
        
        optimizer.zero_grad()
        reconstructed_data = ae_model(data)
        loss = criterion(reconstructed_data, data) 
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.size(0)
    
    avg_loss = total_loss / len(normal_features)
    if (epoch + 1) % 10 == 0 or epoch == 0:
          print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {avg_loss:.6f}")

# Save the trained Autoencoder model
torch.save(ae_model.state_dict(), 'autoencoder_anomaly_detector.pth')
print("✅ Autoencoder model saved as 'autoencoder_anomaly_detector.pth'.")

# ----------------------------------------------------------------------

# --- 5. Determine Anomaly Threshold (τ) ---

ae_model.eval()
normal_losses = []

print("\n--- 5. Calculating Anomaly Threshold (τ) ---")
with torch.no_grad():
    for (data,) in tqdm(feature_loader, desc="Calculating Threshold"):
        # data is already on GPU
        reconstructed_data = ae_model(data)
        # Calculate L2 Reconstruction Loss for each sample
        # OPTIMIZATION 4: Use a faster way to calculate the L2 norm (squared error)
        # You were doing: torch.sum((data - reconstructed_data)**2, dim=1)
        # Using the faster torch.linalg.norm is more direct and potentially optimized.
        sample_losses = torch.linalg.norm(data - reconstructed_data, dim=1).pow(2).cpu().numpy()
        normal_losses.extend(sample_losses)

# Use the 99.9th percentile to set a robust threshold
FINAL_THRESHOLD = np.percentile(np.array(normal_losses), 99.9)

print("\n" + "=" * 50)
print(f"🚨 ANOMALY DETECTION TRAINING COMPLETE 🚨")
print(f"FINAL ANOMALY THRESHOLD (τ): {FINAL_THRESHOLD:.4f}")
print("==================================================")


--- 4. Training Autoencoder for Anomaly Detection ---


Epoch 1/200: 100%|██████████| 42/42 [00:00<00:00, 183.52it/s]


Epoch [1/200], Loss: 0.282793


Epoch 10/200: 100%|██████████| 42/42 [00:00<00:00, 342.33it/s]


Epoch [10/200], Loss: 0.070333


...

Epoch [190/200], Loss: 0.024188


Epoch 200/200: 100%|██████████| 42/42 [00:00<00:00, 322.18it/s]


Epoch [200/200], Loss: 0.023920
✅ Autoencoder model saved as 'autoencoder_anomaly_detector.pth'.

--- 5. Calculating Anomaly Threshold (τ) ---


Calculating Threshold: 100%|██████████| 42/42 [00:00<00:00, 421.96it/s]


🚨 ANOMALY DETECTION TRAINING COMPLETE 🚨
FINAL ANOMALY THRESHOLD (τ): 145.2529


------------- Testing ----------------